# DICOM to Video


## Define Method

In [1]:
import pydicom
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as display
import os
import time
import cv2

def load_dcm(path):
    dcm = pydicom.dcmread(path)
    return dcm

def cvt_bgr(dcm):
    frames = dcm.pixel_array
    if dcm.PhotometricInterpretation == 'YBR_FULL_422':
        for i, f in enumerate(frames):
            frames[i] = cv2.cvtColor(f, cv2.COLOR_YCrCb2BGR)
    elif dcm.PhotometricInterpretation == 'RGB':
        for i, f in enumerate(frames):
            frames[i] = cv2.cvtColor(f, cv2.COLOR_RGB2BGR)
    return frames

def generate_video(frames, fps, output_path, name):
    height, width = frames.shape[1], frames.shape[2]
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    file_format = '.avi'
    output_file = f"{output_path}/{name}{file_format}"
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
    for frame in frames:
        video.write(frame)
    video.release()
    
def save_imgs(save_dir, frames, name):
    os.makedirs(f"{save_dir}/{name}", exist_ok=True)
    for i, img in enumerate(frames):
        cv2.imwrite(f"{save_dir}/{name}/{i}.jpg", img)

def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None
    return cap

def show_video(video_path):
    cap = load_video(video_path)
    if cap is None:
        print("Error: Could not open video.")
        return
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if width < 200 or height < 200:
        width *= 2
        height *= 2
    wImg = widgets.Image(width=width, height=height)
    display.display(wImg)
    if cap.isOpened():
        ret, frame = cap.read()
        while ret:
            tmpStream = cv2.imencode(".jpeg", frame)[1].tobytes()
            wImg.value = tmpStream
            time.sleep(1/fps)
            ret, frame = cap.read()
    cap.release()
    cv2.destroyAllWindows()

In [2]:
import pydicom
from tqdm import tqdm
import pandas as pd

codebook = pd.read_csv('../dataset/RVENet/codebook.csv')
file_format = '.dcm'
dicom_dir = '../dataset/RVENet/DICOM'

pbar = tqdm(codebook.iloc, total=len(codebook), desc='Converting dicom to videos and images', ascii=" =", leave=True)
for code in pbar:
    name = code['FileName']
    fps = code['FPS']
    file_path = f'{dicom_dir}/{name}{file_format}'
    dcm = load_dcm(file_path)
    frames = cvt_bgr(dcm)
    os.makedirs(f'../dataset/RVENet/Videos', exist_ok=True)
    generate_video(frames, fps, '../dataset/RVENet/Videos', name)
    save_imgs('../dataset/RVENet/Images', frames, name)
pbar.close()

Converting dicom to videos and images: 100%|==========| 3583/3583 [1:36:10<00:00,  1.61s/it]


In [38]:
import numpy as np

file_names = codebook['FileName'].values
name = np.random.choice(file_names)
show_video(f'../dataset/RVENet/Videos/{name}.avi')

Image(value=b'', height='708', width='1016')